In [7]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

In [8]:
data = pd.read_csv('/datasets/Electrical_Grid_Stability.csv', sep=';')

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(columns=['stability']), 
    data.stability, 
    test_size=0.3, 
    shuffle=True,
    random_state=42)

X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.FloatTensor(y_train.values)
y_test = torch.FloatTensor(y_test.values)

In [9]:
def init_weights(layer):
    if type(layer) == nn.Linear: # Проверка, что слой — полносвязный
        nn.init.normal_(layer.weight, mean=0, std=1)
        nn.init.normal_(layer.bias, mean=0, std=1)

In [10]:
n_in_neurons = 12
n_hidden_neurons_1 = 6
n_hidden_neurons_2 = 2
n_hidden_neurons_3 = 6
n_out_neurons = 1 

net = nn.Sequential(
    nn.Linear(n_in_neurons, n_hidden_neurons_1),
    nn.Tanh(),
    nn.Linear(n_hidden_neurons_1, n_hidden_neurons_2),
    nn.ReLU(),
    nn.Linear(n_hidden_neurons_2, n_hidden_neurons_3),
    nn.ReLU(),
    nn.Linear(n_hidden_neurons_3, n_out_neurons), 
    nn.Sigmoid()
)

net.apply(init_weights) 

Sequential(
  (0): Linear(in_features=12, out_features=6, bias=True)
  (1): Tanh()
  (2): Linear(in_features=6, out_features=2, bias=True)
  (3): ReLU()
  (4): Linear(in_features=2, out_features=6, bias=True)
  (5): ReLU()
  (6): Linear(in_features=6, out_features=1, bias=True)
  (7): Sigmoid()
)

In [11]:
optimizer = torch.optim.AdamW(net.parameters(), lr=1e-3)

loss = nn.BCELoss()

num_epochs = 100

In [12]:
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    preds = net.forward(X_train).flatten()
            
    loss_value = loss(preds, y_train)

    loss_value.backward()
            
    optimizer.step()

    if epoch % 10 == 0 or epoch == num_epochs - 1:
        net.eval()
        test_preds = net.forward(X_test)
        accuracy = (torch.round(test_preds) == y_test).float().mean().data
        print(accuracy)

tensor(0.4605)
tensor(0.4824)
tensor(0.5030)
tensor(0.5219)
tensor(0.5383)
tensor(0.5540)
tensor(0.5679)
tensor(0.5795)
tensor(0.5894)
tensor(0.5958)
tensor(0.6004)
